In [ ]:
import re
import math
import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

# Hunalign

### Text preparation

English text:

In [ ]:
with open('en.txt', 'r', encoding='utf-8-sig') as f:
    en_text = f.read()

In [ ]:
en_ready = []
for i in sent_tokenize(en_text):
    en_ready.append('<p>')
    en_ready.append(i)

In [ ]:
with open('en_hunalign.txt', 'w', encoding='utf-8-sig') as f:
    f.write('\n'.join(en_ready))

Russian text:

In [ ]:
with open('ru.txt', 'r', encoding='utf-8-sig') as f:
    ru_text = f.read()

In [ ]:
ru_ready = []
for i in sent_tokenize(ru_text):
    new_i = re.sub('([.,!?();])', r' \1 ', i)
    new_i = re.sub('\s{2,}', ' ', new_i)
    ru_ready.append('<p>')
    ru_ready.append(new_i)

In [ ]:
with open('ru_hunalign.txt', 'w', encoding='utf-8-sig') as f:
    f.write('\n'.join(ru_ready))

### Alignment

В папку с приложением hunalign.exe и нулевым словарем null.dic (так как для русского языка нет готового словаря) положить файлы en_demo и ru_demo, в которых находятся в готовом для элайнмента формате предложения на соответствующих языках, открыть в ней командную строку и ввести следующее (результат будет в файле hunalign.txt):

hunalign.exe null.dic en_hunalign.txt ru_hunalign.txt -text -utf -realign > hunalign.txt

### Hunalign table

In [ ]:
with open('hunalign.txt', 'r', encoding='utf-8-sig') as f:
    aligned_text = f.read()

In [ ]:
with_p = aligned_text.split('\n')

without_p = []
for i in range(1, len(with_p), 2):
    without_p.append(with_p[i])

en_hunalign_df = []
for element in without_p:
    for en in range(0, len(element.split('\t')), 3):
        en_hunalign_df.append(element.split('\t')[en])

ru_hunalign_df = []
for element in without_p:
    for ru in range(1, len(element.split('\t')), 3):
        ru_hunalign_df.append(element.split('\t')[ru])

score = []
for element in without_p:
    for sc in range(2, len(element.split('\t')), 3):
        score.append(element.split('\t')[sc])

In [ ]:
df_hunalign = pd.DataFrame({'en': en_hunalign_df, 'ru': ru_hunalign_df, 'score': score})

df_hunalign.to_excel('./hunalign.xlsx', index=False)

### Hunalign table with entities

In [ ]:
points = []

for count, letter in enumerate(en_text):
    if en_text[count-1]+letter == ' .' or letter == '?' or letter == '!':
        points.append(count)

for i, value in enumerate(en_hunalign_df):
    if value == '':
        points.insert(i, '')

In [ ]:
df_entities = pd.read_excel('entities.xlsx', index_col=None, header=None)

In [ ]:
df_hunalign_entities = pd.DataFrame({'en': en_hunalign_df, 'ru': ru_hunalign_df, 'points': points, 'entities': ''*len(points)})

In [ ]:
entities = [[] for _ in range(len(points))]

for i, value in enumerate(df_entities[1]):
    for ind, p in enumerate(points):
        if p != '' and int(value.split(' ')[1]) < int(p):
            entities[[df_hunalign_entities[df_hunalign_entities['points']==p].index.values][0][0]].append(str(value.split(' ')[0])+'-'+df_entities[2][i])
            break

In [ ]:
df_hunalign_entities = pd.DataFrame({'en': en_hunalign_df, 'ru': ru_hunalign_df, 'entities': entities})

df_hunalign_entities.to_excel('./hunalign_entities.xlsx', index=False)

# Fast_align

### Text preparation

In [ ]:
tokenized_en_text = []
for sent in sent_tokenize(en_text.lower()):
    tokenized_en_text.append(tokenizer.tokenize(sent))
    
clean_en_text = []
for sent in tokenized_en_text:
    clean_en_text.append(' '.join(sent))

In [ ]:
tokenized_ru_text = []
for sent in sent_tokenize(ru_text.lower(), language="russian"):
    tokenized_ru_text.append(tokenizer.tokenize(sent))

clean_ru_text = []
for sent in tokenized_ru_text:
    clean_ru_text.append(' '.join(sent))

In [ ]:
with open('en_fastalign.txt', 'w', encoding='utf-8-sig') as f:
    f.write('\n'.join(clean_en_text))

with open('ru_fastalign.txt', 'w', encoding='utf-8-sig') as f:
    f.write('\n'.join(clean_ru_text))

### Alignment

Через консоль Ubuntu. Сначала приведем в нужный формат по предложениям:

paste en_fastalign.txt ru_fastalign.txt | sed "s/$(printf '\t')/ ||| /g" > source_targets.fastalign

Сначала мы рассматриваем английский язык как мишень (target), а русский язык как источник (source), поэтому используем reverse:

./fast_align -i source_targets.fastalign -d -o -v -r > reverse.align

### Fast_align table

In [ ]:
with open('fastalign_en_ru.txt', 'r', encoding='utf-8-sig') as f:
    aligned_words = f.read()

aligned_words = aligned_words.split('\n')

In [ ]:
sootnosh = []
for i in range(len(aligned_words)-1):
    sootnosh.append([[aligned_words[i]],[clean_en_text[i], clean_ru_text[i]]])

In [ ]:
df_fastalign_incomplete = pd.DataFrame({'en': [], 'ru': []}) #поменять местами для ru_en

k = 0
for soot in sootnosh:
    indexes = soot[0][0].split(' ')
    list_en = []
    list_ru = []
    
    for i in indexes:
        if soot[0][0] == '':
            for j in range(len(soot[1][0].split(' '))):
                list_en.append(soot[1][0].split(' ')[j])
                list_ru.append('')
        else:
            list_en.append(soot[1][0].split(' ')[int(i.split('-')[0])])
            list_ru.append(soot[1][1].split(' ')[int(i.split('-')[1])])     
        
    inde = [k]
    df2 = pd.DataFrame(list(zip(inde, inde)), columns=['en', 'ru'])
    df_fastalign_incomplete = df_fastalign_incomplete.append(df2)
    df2 = pd.DataFrame(list(zip(list_en, list_ru)), columns=['en', 'ru']) # поменять местами list_en и list_ru для ru_en и columns
    df_fastalign_incomplete = df_fastalign_incomplete.append(df2)
    k += 1

### Fast_align complete table

In [ ]:
en_fastalign_df = []
for i in range(len(en_hunalign_df)):
    new_str = str(i) + ' '
    en_fastalign_df.append(new_str + en_hunalign_df[i])

sentences = []
for i in en_fastalign_df:
    new_i = i.split(' ')
    sentences.append(new_i)

In [ ]:
incomplete_sentences = []
incomplete_one_sent = []

len_k = int(sentences[-1][0]) + 1
k = 0
for i in df_fastalign_incomplete['en']:
    if i == k + 1:
        incomplete_sentences.append(incomplete_one_sent)
        incomplete_one_sent = []
        k += 1
        incomplete_one_sent.append(i)
    else:
        incomplete_one_sent.append(str(i))
    
incomplete_sentences.append(incomplete_one_sent)
incomplete_sentences[0][0] = 0

In [ ]:
pair = []
incomplete = []

k = 1
minus_len = 0
for i, value in enumerate(df_fastalign_incomplete['ru']):
    if value == k:
        incomplete.append(pair)
        pair = []
        minus_len += len(incomplete[k-1])
        k += 1
        
    pair.append(str(incomplete_sentences[k-1][i-minus_len]) + '-' + str(value))

incomplete.append(pair)
incomplete[0][0] = '0-0'

In [ ]:
complete = []

for one_sent in sentences:
    j = 0
    pair = sentences.index(one_sent)
    
    for i in range(len(one_sent)):
        
        try:
            if one_sent[i].lower() == incomplete[pair][j].split('-')[0]:
                complete.append(incomplete[pair][j])
                j += 1
            elif one_sent[i].lower() == "'s" or one_sent[i].lower() == "'d":
                complete.append(incomplete[pair][j])
                j += 1
            else:
                complete.append(one_sent[i].lower() + '-' + ' ')
        
        except IndexError:
            complete.append(one_sent[i].lower() + '-' + ' ')

In [ ]:
en_fastalign_complete = []
ru_fastalign_complete = []

k = 0
for pair in complete:
    if pair.split('-')[0] == str(k) and pair.split('-')[1] == str(k):
        en_fastalign_complete.append(int(pair.split('-')[0]))
        ru_fastalign_complete.append(int(pair.split('-')[1]))
        k += 1
    else:
        en_fastalign_complete.append(pair.split('-')[0])
        ru_fastalign_complete.append(pair.split('-')[1])

In [ ]:
df_fastalign_complete = pd.DataFrame({'en': en_fastalign_complete, 'ru': ru_fastalign_complete})
df_fastalign_complete.to_excel('./fastalign_en_ru_complete.xlsx', index=False)

In [ ]:
# для ru_en
df_fastalign_complete = pd.DataFrame({'ru': ru_fastalign_complete, 'en': en_fastalign_complete})
df_fastalign_complete.to_excel('./fastalign_ru_en_complete.xlsx', index=False)

### Fast_align table with entities

In [ ]:
all_tags = []

k = 0
full_word = ''
for i, word in enumerate(df_fastalign_complete['en']):
    
    if type(word) != str and math.isnan(word):
        word = ''
        len_token = 2
    elif word == 's' or word == 'd':
        len_token = 2
    else:
        len_token = len(str(word))
   
    try:
        k += len_token + 1
        if (word == '.' or word == '!' or word == '?') and (len(str(df_fastalign_complete['en'][i + 1])) == 1 and str(df_fastalign_complete['en'][i + 1]).isdigit()):
            k -= 2
        elif (word == '.' or word == '!' or word == '?') and (len(str(df_fastalign_complete['en'][i + 1])) == 2 and str(df_fastalign_complete['en'][i + 1]).isdigit()):
            k -= 3 
        elif (word == '.' or word == '!' or word == '?') and (len(str(df_fastalign_complete['en'][i + 1])) == 3 and str(df_fastalign_complete['en'][i + 1]).isdigit()):
            k -= 4

        all_tags.append(str(word) + '-' + ' ')
        for start in df_entities[1]:
            if k - 2 > int(start.split(' ')[1]) and k - 2 <= int(start.split(' ')[2])+1 and word != '.' and word != '!' and word != '?':
                fin = str(word) + '-' + start.split(' ')[0]
                
                if all_tags[len(all_tags) - 1].split('-')[0] == word and all_tags[len(all_tags) - 1].split('-')[1] == ' ':
                    all_tags[len(all_tags) - 1] = fin
                elif all_tags[len(all_tags) - 1].split('-')[0] == word and all_tags[len(all_tags) - 1].split('-')[1] != ' ':
                    all_tags[len(all_tags) - 1] = all_tags[len(all_tags) - 1] + '-' + start.split(' ')[0]
                else:
                    all_tags.append(fin)
                    
    except KeyError:
        break

In [ ]:
en_fastalign_entities = []
fastalign_entities = []
for i, value in enumerate(all_tags):
    
    en_fastalign_entities.append(value.split('-')[0])
    if value.split('-')[1] == ' ':
        fastalign_entities.append('')
    else:
        fastalign_entities.append(value.split('-')[1:])

ru_fastalign_entities = list(df_fastalign_complete['ru'])
en_fastalign_entities.append('.')
fastalign_entities.append('')

In [ ]:
df_fastalign_entities = pd.DataFrame({'en': en_fastalign_entities, 'ru': ru_fastalign_entities, 'tags': fastalign_entities})
df_fastalign_entities.to_excel('./fastalign_en_ru_entities.xlsx', index=False)

### Ru_en fast_align

Теперь проделываем то же самое, но рассматриваем русский язык как мишень (target), а английский как источник (source), используем forward:

./fast_align -i source_targets.fastalign -d -o -v > forward.align

In [ ]:
with open('fastalign_ru_en.txt', 'r', encoding='utf-8-sig') as f:
    aligned_words = f.read()

aligned_words = aligned_words.split('\n')

In [ ]:
dict_tags = {}
list_dict_tags = []

k = 1
for i, word in enumerate(df_fastalign_entities['en']):
    
    if word == str(k) and df_fastalign_entities['ru'][i] == k:
        list_dict_tags.append(dict_tags)
        dict_tags = {}
        k += 1
        
    elif df_fastalign_entities['tags'][i] in dict_tags.keys() and type(df_fastalign_entities['tags'][i]) == str:
        dict_tags[df_fastalign_entities['tags'][i]].append(word)
        
    elif type(df_fastalign_entities['tags'][i]) == str:
        dict_tags[df_fastalign_entities['tags'][i]] = [word]

Проделать sootnosh для ru_en.

In [ ]:
ru_fastalign_entities = []
en_fastalign_entities = []
fastalign_entities = []

k = 0
for i, word in enumerate(df_fastalign_incomplete['en']):
    ru_fastalign_entities.append(df_fastalign_incomplete['ru'][i])
    en_fastalign_entities.append(word)
    
    if word == k + 1 and df_fastalign_incomplete['ru'][i] == k + 1:
        k += 1
    
    for key, value in list_dict_tags[k].items():
        for v in value:
            if word == v:
                fastalign_entities.append(key)
                break
                
    if len(en_fastalign_entities) > len(fastalign_entities):
        fastalign_entities.append('')
if len(en_fastalign_entities) > len(fastalign_entities):
    fastalign_entities.append('')

In [ ]:
df_fastalign_entities = pd.DataFrame({'ru': ru_fastalign_entities, 'en': en_fastalign_entities, 'tags': fastalign_entities})
df_fastalign_entities.to_excel('./fastalign_ru_en_entities.xlsx', index=False)

# Models

### Preparation

In [126]:
df_fastalign_entities = pd.read_excel('fastalign_ru_en_entities.xlsx', index_col=None)

In [129]:
dictionary = []

k = 0
for i, line in enumerate(df_fastalign_entities['ru']):

    if str(df_fastalign_entities['tags'][i]) == 'nan':
        tag = ''
    else:
        tag = df_fastalign_entities['tags'][i]
        
    if line == k and df_fastalign_entities['en'][i] == k:
        dictionary.append(str(k) + '-' + str(k) + '-' + tag)
        k += 1
    else:
        dictionary.append(str(line) + '-' + str(df_fastalign_entities['en'][i]) + '-' + tag)

In [130]:
for_models = ''
for i in ru_hunalign_df:
    for_models += i + ' '
    
for_models = for_models.replace('?', '.')
for_models = for_models.replace('!', '.')
for_models = for_models.replace('~~~', '')
for_models = for_models.replace('. . .', '')

new_for_models = []
k = 0
for i in for_models.split('.'):
    new_i = re.sub(r'[.«»,"\'?:!;—]', '', i)
    new_i = re.sub(r'[-]', ' ', new_i)
    new_i = re.sub(r'[\n\xa0]', ' ', new_i)
    new_i = re.sub(r'  ', ' ', new_i)
    if k == 0:
        new_for_models.append(str(k) + ' ' + new_i)
    else:
        new_for_models.append(str(k) + new_i)
    k += 1

NameError: name 'ru_hunalign_df' is not defined

In [ ]:
for_models = ''
for i in new_for_models:
    for_models += '' + i

In [137]:
df_fastalign_entities_complete = []

k = 0
j = 0
for word in for_models.split(' '):
    
    if word.lower() != dictionary[j].split('-')[0]:
        df_fastalign_entities_complete.append(word.lower() + '-' + '' + '-' + '')
        continue
        
    else:
        df_fastalign_entities_complete.append(dictionary[j].split('-')[0] + '-' + dictionary[j].split('-')[1] + '-' + dictionary[j].split('-')[2])
        j += 1
        continue

### Natasha Slovnet

In [241]:
from navec import Navec
from slovnet import NER

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

natasha_slovnet = ner(for_models)

In [242]:
df_natasha = []

length = 0
for i in range(len(df_fastalign_entities_complete)):
    length += len(df_fastalign_entities_complete[i].split('-')[0]) + 1
    
    for j in natasha_slovnet.spans:
        tag = ''
        if length >= j.start and length <= j.stop:
            tag = j.type
            break
    
    try:
        new_tag = df_fastalign_entities_complete[i + 1] + '-' + tag
        df_natasha.append(new_tag)
    except IndexError:
        break

In [263]:
ru_full_tags = []
en_full_tags = []
en_tag_full_tags = []
natasha = []

for i in df_natasha:
    ru_full_tags.append(i.split('-')[0])
    en_full_tags.append(i.split('-')[1])
    en_tag_full_tags.append(i.split('-')[2][2:5])
    natasha.append(i.split('-')[3])

### Stanza

In [118]:
import stanza

nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
doc_stanza = nlp(for_models)

2023-05-18 16:02:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-05-18 16:02:40 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2023-05-18 16:02:40 INFO: Use device: cpu
2023-05-18 16:02:40 INFO: Loading: tokenize
2023-05-18 16:02:41 INFO: Loading: ner
2023-05-18 16:02:45 INFO: Done loading processors!


In [138]:
df_stanza = []

length = 0
for i in range(len(df_fastalign_entities_complete)):
    length += len(df_fastalign_entities_complete[i].split('-')[0]) + 1

    for sent in doc_stanza.sentences:
        for ent in sent.ents:
            tag = ''
            if length >= ent.start_char and length <= ent.end_char:
                tag = ent.type
                break
        break
    
    try:
        new_tag = df_fastalign_entities_complete[i + 1] + '-' + tag
        df_stanza.append(new_tag)
    except IndexError:
        break

In [239]:
stanza = []
for i in df_stanza:
    stanza.append(i.split('-')[3])

### Deeppavlov RuBert

In [136]:
for_models = '0 Однажды давным давно в старое доброе время шла по дороге коровушка Му му шла и шла и встретила на дороге хорошенького прехорошенького мальчика а звали его Бу бу Папа рассказывал ему эту сказку папа смотрел на него через стеклышко 1 У него было волосатое лицо 2 Он был мальчик Бу бу 3 Му му шла по дороге где жила Бетти Берн она продавала лимонные леденцы 4 О цветы дикой розыНа зеленом лугу 5 Он пел эту песню 6 Это была его песня 7 О таритатам лозы Когда намочишь в постельку сначала делается горячо а потом холодно 8 Мама подкладывает клеенку 9 От нее такой чудной запах 10 От мамы пахнет приятнее чем от папы 11 Она играет ему на рояле матросский танец чтобы он плясал 12 Он плясал Тра ля ля ля ля Тра ля ля тра ля ля ди Тра ля ля ля ля Тра ля ля ля ля Дядя Чарльз и Дэнти хлопали в ладоши 13 Они старее папы и мамы но дядя Чарльз еще старее Дэнти 14 У Дэнти в шкафу две щетки 15 Щетка с коричневой бархатной спинкой в честь Майкла Дэвитта а щетка с зеленой бархатной спинкой в честь Парнелла 16 Дэнти давала ему мятный леденец всякий раз когда он приносил ей бумажную салфетку 17 Вэнсы жили в доме семь 18 У них другие папы и мамы 19 Это папа и мама Эйлин 20 Когда они вырастут большие он женится на Эйлин 21 Он спрятался под стол 22 Мама сказала 23 Проси прощенья Стивен 24 Дэнти сказала  А не попросишь прилетит орел и выклюет тебе глаза 25 И выклюет тебе глаза Проси прощенья егоза Проси прощенья егоза И выклюет тебе глаза Проси прощенья егоза И выклюет тебе глаза И выклюет тебе глаза Проси прощенья егоза 26 * На больших спортивных площадках толпились мальчики 27 Все кричали и воспитатели их громко подбадривали 28 Вечерний воздух был бледный и прохладный и после каждой атаки и удара футболистов лоснящийся кожаный шар как тяжелая птица взлетал в сером свете 29 Он топтался в самом хвосте своей команды подальше от воспитателя подальше от грубых ног и время от времени делал вид что бегает 30 Он чувствовал себя маленьким и слабым среди толпы играющих и глаза у него были слабые и слезились 31 Роди Кикем не такой он будет капитаном третьей команды говорили мальчики 32 Роди Кикем хороший мальчик а Вонючка Роуч противный 33 У Роди Кикема щитки для ног в шкафу в раздевалке и корзинка со сладостями в столовой 34 У Вонючки Роуча огромные руки 35 Он говорит что постный пудинг это месиво в жиже 36 А как то раз он спросил  Как тебя зовут 37 Стивен ответил  Стивен Дедал 38 А Вонючка Роуч сказал  Что это за имя 39 И когда Стивен не нашелся что ответить Вонючка Роуч спросил  Кто твой отец 40 Стивен ответил  Джентльмен 41 Тогда Вонючка Роуч спросил  А он не мировой судья 42 Он топтался в самом хвосте своей команды делая иногда короткие перебежки 43 Руки его посинели от холода 44 Он засунул их в боковые карманы своей серой подпоясанной куртки 45 Пояс это такая штука над карманами 46 А вот в драке о тех кто победил говорят за пояс заткнул 47 Как то один мальчик сказал Кэнтуэллу 48 Я бы тебя мигом за пояс заткнул 49 А Кэнтуэлл ответил 50 Попробуй ка Сесила Сандера за пояс заткнуть 51 Я посмотрю как он тебе даст под зад 52 Так некрасиво выражаться 53 Мама сказала чтобы он не водился с грубыми мальчиками в колледже 54 Мама такая красивая 55 В первый день в приемной замка она когда прощалась с ним слегка подняла свою вуаль чтобы поцеловать его и нос и глаза у нее были красные 56 Но он притворился будто не замечает что она сейчас расплачется 57 Мама красивая но когда она плачет она уже не такая красивая 58 А папа дал ему два пятишиллинговика пусть у него будут карманные деньги 59 И папа сказал чтобы он написал домой если ему что нибудь понадобится и чтобы он ни в коем случае не ябедничал на товарищей 60 Потом у двери ректор пожал руки папе и маме и сутана его развевалась на ветру а коляска с папой и мамой стала отъезжать 61 Они махали руками и кричали ему из коляски Прощай Стивен прощай 62 Прощай Стивен прощай 63 Вокруг него началась свалка из за мяча и страшась этих горящих глаз и грязных башмаков он нагнулся и стал смотреть мальчикам под ноги 64 Они дрались пыхтели и ноги их топали толкались и брыкались 65 Потом желтые ботинки Джека Лотена наподдали мяч и все другие ботинки и ноги ринулись за ним 66 Он пробежал немножко и остановился 67 Не стоило бежать 68 Скоро все поедут домой 69 После ужина в классе он переправит число приклеенное у него в парте с семидесяти семи на семьдесят шесть 70 Лучше бы сейчас быть в классе чем здесь на холоде 71 Небо бледное и холодное а в главном здании в замке огни 72 Он думал из какого окна Гамильтон Роуэн бросил свою шляпу на изгородь и были ли тогда цветочные клумбы под окнами 73 Однажды когда он был в замке тамошний служитель показал ему следы солдатских пуль на двери и дал ореховый сухарик какие едят в общине 74 Как хорошо и тепло смотреть на огни в замке 75 Совсем как в книжке 76 Может быть Лестерское аббатство было такое 77 А какие хорошие фразы были в учебнике д ра Корнуэлла 78  Они похожи на стихи но это только примеры чтобы научиться писать правильно Уолси умер в Лестерском аббатстве Где погребли его аббаты Растения съедают черви Животных съедает рак Хорошо бы лежать сейчас на коврике у камина подперев голову руками и думать про себя об этих фразах 79 Он вздрогнул будто по телу пробежала холодная липкая вода 80 Подло было со стороны Уэллса столкнуть его в очко уборной за то что он не захотел обменять свою маленькую табакерку на игральную кость которой Уэллс выиграл сорок раз в бабки 81 Какая холодная и липкая была вода 82 А один мальчик раз видел как большая крыса прыгнула в жижу 83 Мама с Дэнти сидели у камина и дожидались когда Бриджет подаст чай 84 Мама поставила ноги на решетку и ее вышитые бисером ночные туфли нагрелись и от них так хорошо и тепло пахло 85 Дэнти знала массу всяких вещей 86 Она учила его где находится Мозамбикский пролив и какая самая длинная река в Америке и как называется самая высокая гора на Луне 87 Отец Арнолл знает больше чем Дэнти потому что он священник но папа и дядя Чарльз оба говорили что Дэнти умная и начитанная женщина 88 А иногда Дэнти делала такой звук после обеда и подносила руку ко рту это была отрыжка 89 Голос с дальнего конца площадки крикнул  Все домой 90 Потом голоса из младших и средних классов подхватили  Домой 91 Все домой 92 Мальчики сходились со всех сторон раскрасневшиеся и грязные и он шагал среди них радуясь что идут домой 93 Роди Кикем держал мяч за скользкую шнуровку 94 Один мальчик попросил поддать еще напоследок но он шел себе и даже ничего не ответил 95 Саймон Мунен сказал чтобы он этого не делал так как на них смотрит надзиратель 96 Тогда тот мальчик повернулся к Саймону Мунену и сказал Мы все знаем почему ты так говоришь 97 Ты известный подлиза 98 Какое странное слово подлиза 99 Мальчик обозвал так Саймона Мунена потому что Саймон Мунен связывал иногда фальшивые рукава на спине надзирателя Макглэйда а тот делал вид что сердится 100 Противный звук у этого слова 101 Однажды он мыл руки в уборной гостиницы на Уиклоу стрит а потом папа вынул пробку за цепочку и грязная вода стала стекать через отверстие в раковине 102 А когда она вся стекла потихоньку отверстие в раковине сделало такой звук длизс 103 Только громче 104 Он вспоминал это и белые стены уборной и ему делалось сначала холодно а потом жарко 105 Ему сделалось сначала холодно а потом чуть чуть жарко 106 И он видел слова напечатанные на кранах 107 В этом что то было чудное 108 В коридоре был тоже холодный воздух 109 Он был сыроватый и чудной 110 Но скоро зажгут газ и он будет тихонечко так петь точно какую то песенку 111 Все одну и ту же и когда мальчики не шумят в рекреационном зале ее слышно 112 Урок арифметики начался 113 Отец Арнолл написал на доске трудный пример и сказал  Ну кто победит 114 Живей Йорк 115 Живей Ланкастер 116 Стивен старался изо всех сил но пример был очень трудный и он сбился 117 Маленький шелковый значок с белой розой приколотый к его куртке на груди начал дрожать 118 Он был не очень силен в арифметике но старался изо всех сил чтобы Йорки не проиграли 119 Отец Арнолл сделал очень строгое лицо но он вовсе не сердился он смеялся 120 Вдруг Джек Лотен хрустнул пальцами и отец Арнолл посмотрел в его тетрадку и сказал Верно 121 Браво Ланкастер 122 Алая роза победила 123 Не отставай Йорк 124 Ну ка поднатужьтесь 125 Джек Лотен поглядывал на них со своего места 126 Маленький шелковый значок с алой розой казался очень нарядным на его синей матроске 127 Стивен почувствовал что его лицо тоже покраснело когда он вспомнил как мальчики держали пари кто будет первым учеником Джек Лотен или он 128 Были недели когда Джек Лотен получал билет первого ученика а были недели когда он получал билет первого ученика 129 Его белый шелковый значок дрожал и дрожал все время пока он решал следующий пример и слушал голос отца Арнолла 130 Потом все его рвение пропало и он почувствовал как лицо у него сразу похолодело 131 Он подумал что оно должно быть стало совсем белым раз так похолодело 132 Он не мог решить пример но это было не важно 133 Белые розы и алые розы какие красивые цвета 134 И билеты первого второго и третьего ученика тоже очень красивые розовые бледно желтые и сиреневые 135 Бледно желтые сиреневые и розовые розы тоже красивые 136 Может быть дикие розы как раз такие и ему вспомнилась песенка о цветах дикой розы на зеленом лугу 137 А вот зеленых роз не бывает'

In [23]:
from deeppavlov import configs, build_model

In [25]:
ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2023-05-18 14:15:53.334 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz to C:\Users\Ирина\.deeppavlov\models\ner_rus_bert_torch_new.tar.gz
2023-05-18 14:15:54.25 WARNING in 'deeppavlov.core.data.utils'['utils'] at line 114: Found a partial download C:\Users\Ирина\.deeppavlov\models\ner_rus_bert_torch_new.tar.gz.part
 56%|████████████████████████████████████████████████▏                                     | 808M/1.44G [00:00<?, ?B/s]2023-05-18 14:15:54.35 WARNING in 'deeppavlov.core.data.utils'['utils'] at line 118: Download stopped abruptly, trying to resume from 807534592 to reach 1440917530
100%|█████████████████████████████████████████████████████████████████████████████| 1.44G/1.44G [09:58<00:00, 1.06MB/s]
2023-05-18 14:25:52.116 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting C:\Users\Ирина\.deeppavlov\models\ner_rus_bert_torch_new.tar.gz archive into C:\Users\Ирина\.d

C:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ирина\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [82]:
n = 512
chunks = [for_models[i:i+n] for i in range(0, len(for_models), n)]

In [51]:
doc_bert = []
for i in chunks:
    doc_bert.append(ner_model([i]))

In [205]:
df_bert = []
for i in range(len(doc_bert)-1):
    for j in range(len(doc_bert[i][0][0])):
        for f in for_models.split(' '):
            if f == doc_bert[i][0][0][j]:
                if doc_bert[i][1][0][j] == 'O':
                    df_bert.append(f + '-' + doc_bert[i][1][0][j])
                else:
                    df_bert.append(f + '-' + doc_bert[i][1][0][j][2:])
                break
            elif f == doc_bert[i][0][0][j] + doc_bert[i+1][0][0][0]:
                df_bert.append(f + '-' + 'O')
                break

# отдельно рассмотреть последнее:
#for i in range(1, len(doc_bert[-1][0][0])):
#    df_bert.append(doc_bert[-1][0][0][i] + '-' + doc_bert[-1][1][0][i])

In [259]:
bert = []
for i in df_bert:
    if i.split('-')[1] == 'O':
        bert.append('')
    else:
        bert.append(i.split('-')[1])

### Dataset with all tags

In [264]:
df_alltags = pd.DataFrame({'ru': ru_full_tags, 'en': en_full_tags, 'en_tag': en_tag_full_tags, 'natasha': natasha, 'stanza': stanza, 'bert': bert})
df_alltags.to_excel('./all_entities.xlsx', index=False)

# Metrics

In [267]:
df_alltags = pd.read_excel('all_entities.xlsx', index_col=None)

In [266]:
from sklearn.metrics import classification_report

In [270]:
en_tag = []
for i in df_alltags['en_tag']:
    if str(i) == 'nan':
        en_tag.append('')
    else:
        en_tag.append(i)

### Natasha

In [278]:
natasha_tag = []
for i in df_alltags['natasha']:
    if str(i) == 'nan':
        natasha_tag.append('')
    else:
        natasha_tag.append(i)

In [294]:
natasha_en_tag = []
natasha_natasha_tag = []
for i in range(len(en_tag)):
    if en_tag[i] == 'GPE':
        natasha_en_tag.append('LOC')
        natasha_natasha_tag.append(natasha_tag[i])
    elif en_tag[i] != '' and natasha_tag[i] != '' and en_tag[i] != 'FAC' and en_tag[i] != 'VEH':
        natasha_en_tag.append(en_tag[i])
        natasha_natasha_tag.append(natasha_tag[i])
    elif (en_tag[i] == '' and natasha_tag[i] != '') or (en_tag[i] != '' and natasha_tag[i] == '' and en_tag[i] != 'FAC' and en_tag[i] != 'VEH'):
        natasha_en_tag.append(en_tag[i])
        natasha_natasha_tag.append(natasha_tag[i])

In [295]:
print(classification_report(natasha_en_tag, natasha_natasha_tag))

              precision    recall  f1-score   support

                   0.00      0.00      0.00        43
         LOC       0.43      0.25      0.32        12
         PER       0.65      0.33      0.44       218

    accuracy                           0.28       273
   macro avg       0.36      0.19      0.25       273
weighted avg       0.54      0.28      0.37       273



### Stanza

In [300]:
stanza_tag = []
for i in df_alltags['stanza']:
    if str(i) == 'nan':
        stanza_tag.append('')
    else:
        stanza_tag.append(i)

In [311]:
stanza_en_tag = []
stanza_stanza_tag = []
for i in range(len(en_tag)):
    if stanza_tag == 'MISC' and en_tag[i] != '':
        stanza_en_tag.append(en_tag[i])
        stanza_stanza_tag.append(en_tag[i])
    elif en_tag[i] == 'GPE':
        stanza_en_tag.append('LOC')
        stanza_stanza_tag.append(stanza_tag[i])
    elif en_tag[i] != '' and stanza_tag[i] != '' and en_tag[i] != 'FAC' and en_tag[i] != 'VEH':
        stanza_en_tag.append(en_tag[i])
        stanza_stanza_tag.append(stanza_tag[i])
    elif (en_tag[i] == '' and stanza_tag[i] != '') or (en_tag[i] != '' and stanza_tag[i] == '' and en_tag[i] != 'FAC' and en_tag[i] != 'VEH'):
        stanza_en_tag.append(en_tag[i])
        stanza_stanza_tag.append(stanza_tag[i])

In [314]:
print(classification_report(stanza_en_tag, stanza_stanza_tag))

              precision    recall  f1-score   support

                   0.00      0.00      0.00        70
         LOC       0.50      0.25      0.33        12
        MISC       0.00      0.00      0.00         0
         PER       0.58      0.34      0.43       218

    accuracy                           0.26       300
   macro avg       0.27      0.15      0.19       300
weighted avg       0.44      0.26      0.33       300



C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Bert

In [315]:
bert_tag = []
for i in df_alltags['bert']:
    if str(i) == 'nan':
        bert_tag.append('')
    else:
        bert_tag.append(i)

In [317]:
bert_en_tag = []
bert_bert_tag = []
for i in range(len(en_tag)):
    if en_tag[i] == 'GPE':
        bert_en_tag.append('LOC')
        bert_bert_tag.append(bert_tag[i])
    elif en_tag[i] != '' and bert_tag[i] != '' and en_tag[i] != 'FAC' and en_tag[i] != 'VEH':
        bert_en_tag.append(en_tag[i])
        bert_bert_tag.append(bert_tag[i])
    elif (en_tag[i] == '' and bert_tag[i] != '') or (en_tag[i] != '' and bert_tag[i] == '' and en_tag[i] != 'FAC' and en_tag[i] != 'VEH'):
        bert_en_tag.append(en_tag[i])
        bert_bert_tag.append(bert_tag[i])

In [321]:
print(classification_report(bert_en_tag, bert_bert_tag))

              precision    recall  f1-score   support

                   0.00      0.00      0.00        33
         LOC       0.50      0.25      0.33        12
         PER       0.68      0.29      0.41       218

    accuracy                           0.25       263
   macro avg       0.39      0.18      0.25       263
weighted avg       0.59      0.25      0.36       263

